In [1]:
import json
import re
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tqdm import tqdm
import pandas as pd


# 去除标点符号的函数
def remove_punctuation(text):
    """
    移除输入文本中的标点符号。
    Args:
    - text: 输入的文本字符串。

    Returns:
    - 去除标点符号后的文本字符串。
    """
    return re.sub(r'[^\w\s]', '', text)


# 最长公共子文本串算法
def algrot(text1, text2):
    """
    使用动态规划算法，计算两个字符串的最长公共子文本串。
    Args:
    - text1: 第一个字符串。
    - text2: 第二个字符串。

    Returns:
    - 最长公共子文本串。
    """
    m = len(text1)
    n = len(text2)
    max_len = 0
    ending_index = m
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # 动态规划填充表格，计算公共子串长度
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                if dp[i][j] > max_len:
                    max_len = dp[i][j]
                    ending_index = i
            else:
                dp[i][j] = 0

    # 返回最长公共子串
    return text1[ending_index - max_len: ending_index]


# 查找文本中的所有最长公共子串
def find_all_common_substrings(text1, text2):
    """
    查找文本1和文本2的所有公共子串，返回一个子串列表。
    Args:
    - text1: 第一个字符串。
    - text2: 第二个字符串。

    Returns:
    - 所有公共子串的列表。
    """
    m = len(text1)
    n = len(text2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    substrings = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                substrings.append(text1[i - dp[i][j]:i])
            else:
                dp[i][j] = 0

    return list(set(substrings))  # 去重


# 计算匹配占比
def calculate_match_ratio(substring, content):
    """
    计算公共子串在目标文本中的占比。
    Args:
    - substring: 公共子串。
    - content: 目标文本。

    Returns:
    - 公共子串长度与目标文本长度的比值。
    """
    return len(substring) / len(content) if content else 0


# 查找最佳匹配
def find_best_match(predict_item, labels):
    """
    在 label 列表中为 predict_item 查找最佳匹配的 label 项。
    Args:
    - predict_item: 需要匹配的 predict 项。
    - labels: label 列表。

    Returns:
    - 最佳匹配的 label 项。
    """
    best_match = None
    highest_ratio = 0

    # 去除标点符号
    predict_content_no_punc = remove_punctuation(predict_item['content'])

    for label in labels:
        label_content_no_punc = remove_punctuation(label['content'])

        # 找到所有公共子串
        common_substrings = find_all_common_substrings(predict_content_no_punc, label_content_no_punc)

        # 过滤公共子串，确保其长度占 predict_content 的70%以上
        valid_substrings = [sub for sub in common_substrings if len(sub) / len(predict_content_no_punc) >= 0.8]

        # 如果没有符合的子串，跳过这次匹配
        if not valid_substrings:
            continue

        # 计算公共子串与目标文本的占比，找出占比最高的
        for substring in valid_substrings:
            ratio = calculate_match_ratio(substring, label_content_no_punc)
            if ratio > highest_ratio:
                highest_ratio = ratio
                best_match = label
    if best_match is None:
        return {"content": "其它", "type": "其它"}

    return best_match


# 处理预测与匹配
def process_predictions(predict, labels):
    """
    处理 predict 列表，找到每个 predict 项在 label 列表中的最佳匹配项。
    Args:
    - predict: 需要处理的 predict 列表。
    - labels: label 列表。

    Returns:
    - 包含匹配结果的 predict 列表。
    """
    results = []

    # 遍历 predict 列表中的所有项
    for predict_item in predict:
        best_match = find_best_match(predict_item, labels)

        # 只要找到匹配的 label 项，不管 type 是否匹配，都进行记录
        if best_match:
            predict_item['matched_content'] = best_match['content']
            predict_item['matched_type'] = best_match['type']

        results.append(predict_item)

    return results


# 提取 filtered_result 中 matched_type 为 "发起" 和 "评价" 的数据
def filter_results(results):
    """
    从 results 中提取 matched_type 为 '发起' 和 '评价' 的项，构造 filtered_result。
    Args:
    - results: 原始匹配结果列表

    Returns:
    - filtered_result: 过滤后的列表，仅包含 '发起' 和 '评价' 类型的项。
    """
    filtered_result = [item for item in results if item['matched_type'] in ['发起', '评价']]
    return filtered_result


# 基于 filtered_result 计算 Precision, Recall, F1, 和 Accuracy
def calculate_metrics(filtered_result):
    """
    计算 '发起' 和 '评价' 两个类别的 Precision, Recall, F1 Score 和 Accuracy。
    Args:
    - filtered_result: 经过过滤的匹配结果列表

    Returns:
    - metrics: 包含 precision, recall, f1 和 accuracy 的字典
    """
    # 提取实际标签（true labels）和预测标签（predicted labels）
    y_true = [item['type'] for item in filtered_result]  # 实际标签
    y_pred = [item['matched_type'] for item in filtered_result]  # 预测标签

    # 计算 precision, recall, f1 和 accuracy
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    accuracy = accuracy_score(y_true, y_pred)

    # 返回计算结果
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    }

    # 定义 label 和 predict 列表


label = [
    {
        "type": "评价",
        "content": "不行，你按照老师的要求去做。"
    },
    {
        "type": "其它",
        "content": "请坐。"
    },
    {
        "type": "评价",
        "content": "要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是在那里随便想讲话就讲话。你坐端正，坐好，快点坐下来。对，没有举手的我也要请。上课，就要积极开动脑筋。像上次开汇总课一样。每个同学都认真举手。"
    },
    {
        "type": "发起",
        "content": "好，往下面来。你是读喜鹊的话啊。喜鹊来啊，喜鹊小喜鹊停来上来来表演啊。忘记了，可以稍微看一下啊。"
    },
    {
        "type": "评价",
        "content": "杏四桃三。请坐啊。虽然她讲的不流畅啊，但是他确实是勇敢的上台。而且他刚刚坐在下面是认真听的啊。最起码比下面不敢举手的小朋友要勇敢。这一点我就要鼓励。"
    },
    {
        "type": "发起",
        "content": "你是喜鹊来。再请只喜鹊。"
    },
    {
        "type": "评价",
        "content": "喜鹊没有刚刚乌鸦讲的好。"
    },
    {
        "type": "发起",
        "content": "对，我要请只非常棒的小喜鹊。嗯，哎，掌声鼓励。"
    },
    {
        "type": "讲解",
        "content": "但是讲小故事的小朋友不是背书，可以加上动作。像刚刚几只小乌鸦就扮演的非常好啊。"
    },
    {
        "type": "发起",
        "content": "好，接下来请来演小杜鹃。好的，你来。嗯，我的书在上面。你实在忘记了，可以看一下提示啊。"
    },
    {
        "type": "评价",
        "content": "你有这个耐心吗？你前面用过了，前面乌鸦已经说过了。请坐好，掌声鼓励一下。小杜鹃有点严肃啊。好啦，这小杜鹃可爱一点，活泼一点。猴哥猴哥，着急一点好不好？"
    },
    {
        "type": "发起",
        "content": "来，谁再来演一演？你来罗若涵。你有这个耐心，谁说过了啊。"
    },
    {
        "type": "发起",
        "content": "你有这个耐心，谁说过了啊。"
    }
]
predict = [

    {
        "type": "发起",
        "content": "对，"
    },
    {
        "type": "发起",
        "content": "没有举手的我也要请。"
    },
    {
        "type": "其它",
        "content": "上课，"
    },
    {
        "type": "讲解",
        "content": "就要积极开动脑筋。"
    },
    {
        "type": "讲解",
        "content": "像上次开汇总课一样。"
    },
    {
        "type": "讲解",
        "content": "每个同学都认真举手。"
    },
    {
        "type": "发起",
        "content": "好，往下面来。"
    },
    {
        "type": "发起",
        "content": "你是读喜鹊的话啊。"
    },
    {
        "type": "发起",
        "content": "喜鹊来啊，喜鹊小喜鹊停来上来来表演啊。"
    },
    {
        "type": "发起",
        "content": "忘记了，可以稍微看一下啊。"
    },
    {
        "type": "讲解",
        "content": "杏四桃三。"
    },
    {
        "type": "其它",
        "content": "请坐啊。"
    },
    {
        "type": "评价",
        "content": "虽然她讲的不流畅啊，"
    },
    {
        "type": "讲解",
        "content": "但是他确实是勇敢的上台。"
    },
    {
        "type": "讲解",
        "content": "而且他刚刚坐在下面是认真听的啊。"
    },
    {
        "type": "讲解",
        "content": "最起码比下面不敢举手的小朋友要勇敢。"
    },
    {
        "type": "评价",
        "content": "这一点我就要鼓励。"
    },
    {
        "type": "发起",
        "content": "你是喜鹊来。"
    },
    {
        "type": "发起",
        "content": "再请只喜鹊。"
    },
    {
        "type": "讲解",
        "content": "喜鹊没有刚刚乌鸦讲的好。"
    },
    {
        "type": "发起",
        "content": "对，我要请只非常棒的小喜鹊。"
    },
    {
        "type": "讲解",
        "content": "嗯，哎，掌声鼓励。"
    },
    {
        "type": "讲解",
        "content": "但是讲小故事的小朋友不是背书，可以加上动作。"
    },
    {
        "type": "讲解",
        "content": "像刚刚几只小乌鸦就扮演的非常好啊。"
    },
    {
        "type": "发起",
        "content": "好，接下来请来演小杜鹃。"
    },
    {
        "type": "发起",
        "content": "好的，你来。"
    },
    {
        "type": "讲解",
        "content": "嗯，我的书在上面。"
    },
    {
        "type": "发起",
        "content": "你实在忘记了，可以看一下提示啊。"
    },
    {
        "type": "讲解",
        "content": "你有这个耐心吗？"
    },
    {
        "type": "讲解",
        "content": "你前面用过了，前面乌鸦已经说过了。"
    },
    {
        "type": "其它",
        "content": "请坐好，"
    },
    {
        "type": "讲解",
        "content": "掌声鼓励一下。"
    },
    {
        "type": "讲解",
        "content": "小杜鹃有点严肃啊。"
    },
    {
        "type": "发起",
        "content": "好啦，这小杜鹃可爱一点，活泼一点。"
    },
    {
        "type": "发起",
        "content": "猴哥猴哥，着急一点好不好？"
    },
    {
        "type": "发起",
        "content": "来，谁再来演一演？"
    },
    {
        "type": "发起",
        "content": "你来罗若涵。"
    },
    {
        "type": "讲解",
        "content": "结果你有这么大耐心吗？"
    },
    {
        "type": "讲解",
        "content": "你有这个耐心，"
    },
    {
        "type": "讲解",
        "content": "谁说过了啊。"
    }
]
results = process_predictions(predict, label)
print(f"results_no_filter: {results}")
# 提取过滤结果
filtered_result = filter_results(results)
print("filtered_result", filtered_result)

In [2]:
    # 执行更新后的处理


results_no_filter: [{'type': '其它', 'content': '不行，', 'matched_content': '不行，你按照老师的要求去做。', 'matched_type': '评价'}, {'type': '讲解', 'content': '你按照老师的要求去做。', 'matched_content': '不行，你按照老师的要求去做。', 'matched_type': '评价'}, {'type': '其它', 'content': '请坐。', 'matched_content': '请坐。', 'matched_type': '其它'}, {'type': '评价', 'content': '要批评。', 'matched_content': '要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是在那里随便想讲话就讲话。你坐端正，坐好，快点坐下来。对，没有举手的我也要请。上课，就要积极开动脑筋。像上次开汇总课一样。每个同学都认真举手。', 'matched_type': '评价'}, {'type': '讲解', 'content': '你等会儿三十秒的话都到我这来。', 'matched_content': '要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是在那里随便想讲话就讲话。你坐端正，坐好，快点坐下来。对，没有举手的我也要请。上课，就要积极开动脑筋。像上次开汇总课一样。每个同学都认真举手。', 'matched_type': '评价'}, {'type': '讲解', 'content': '一定要积极思考，', 'matched_content': '要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是在那里随便想讲话就讲话。你坐端正，坐好，快点坐下来。对，没有举手的我也要请。上课，就要积极开动脑筋。像上次开汇总课一样。每个同学都认真举手。', 'matched_type': '评价'}, {'type': '讲解', 'content': '不是自己坐在那里。', 'matched_content': '要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读

In [3]:
df = pd.DataFrame(filtered_result)
df

,type,content,matched_content,matched_type
0,其它,不行，,不行，你按照老师的要求去做。,评价
1,讲解,你按照老师的要求去做。,不行，你按照老师的要求去做。,评价
2,评价,要批评。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
3,讲解,你等会儿三十秒的话都到我这来。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
4,讲解,一定要积极思考，,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
5,讲解,不是自己坐在那里。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
6,讲解,想不读就不读。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
7,讲解,也不是在那里随便想讲话就讲话。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
8,其它,你坐端正，,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
9,其它,坐好，,你有这个耐心吗？你前面用过了，前面乌鸦已经说过了。请坐好，掌声鼓励一下。小杜鹃有点严肃啊。好...,评价


In [65]:
def remove_punctuation(text):
    """
    移除输入文本中的标点符号。
    Args:
    - text: 输入的文本字符串。

    Returns:
    - 去除标点符号后的文本字符串。
    """
    return re.sub(r'[^\w\s]', '', text)


# 查找文本中的所有最长公共子串
def find_all_common_substrings(text1, text2):
    """
    查找文本1和文本2的所有公共子串，返回一个子串列表。
    Args:
    - text1: 第一个字符串。
    - text2: 第二个字符串。

    Returns:
    - 所有公共子串的列表。
    """
    m = len(text1)
    n = len(text2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    substrings = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                substrings.append(text1[i - dp[i][j]:i])
            else:
                dp[i][j] = 0

    return list(set(substrings))  # 去重


def find_best_match_expanded_with_results(predict_items, labels):
    """
    在 label 列表中为多个 predict_item 查找最佳匹配的 label 项，并将匹配结果保存到 predict_items 中。
    返回包含匹配结果的 predict_items 列表。
    """
    start_index = 0  # 初始化从第一个 label 开始

    for predict_item in predict_items:
        best_match = None
        highest_ratio = 0
        best_match_index = -1  # 用于记录匹配的索引
        
        # 去除标点符号
        predict_content_no_punc = remove_punctuation(predict_item['content'])

        # 只在 start_index 到 start_index + 5 的范围内进行匹配
        for i in range(start_index, min(start_index + 5, len(labels))):
            label = labels[i]
            label_content_no_punc = remove_punctuation(label['content'])

            # 找到所有公共子串
            common_substrings = find_all_common_substrings(predict_content_no_punc, label_content_no_punc)

            # 放宽公共子串长度的限制，确保匹配的更宽松
            valid_substrings = [sub for sub in common_substrings if len(sub) / len(predict_content_no_punc) >= 0.3]

            # 如果没有符合的子串，跳过这次匹配
            if not valid_substrings:
                continue

            # 计算公共子串与目标文本的占比，找出占比最高的
            for substring in valid_substrings:
                ratio = calculate_match_ratio(substring, label_content_no_punc)
                if ratio > highest_ratio:
                    highest_ratio = ratio
                    best_match = label
                    best_match_index = i  # 记录匹配成功的索引

        if best_match is None:
            predict_item['best_match'] = {"content": "其它", "type": "其它"}
        else:
            predict_item['best_match'] = best_match

            # 如果匹配的索引更新了，才更新 start_index
            if best_match_index != -1 and best_match_index > start_index:
                start_index = best_match_index  # 更新起始索引为匹配成功的索引

    return predict_items



In [66]:
label = [
{"type": "发起", "content": "请袁艺喊“开始上课”，声音要响亮啊。"},
{"type": "其它", "content": "同学们好，请坐。"},
{"type": "评价", "content": "真棒。"},
{"type": "讲解", "content": "昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。"},
{"type": "发起", "content": "我们来跟他打打招呼。“猴子”。"},
{"type": "讲解", "content": "昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？"},
{"type": "发起", "content": "好，现在用眼睛看，用心记住这个字，“猴子”。"},
{"type": "讲解", "content": "哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。"},
{"type": "发起", "content": "它分别种了什么树呢？谁来说说？于凯，你来说说看。"},
{"type": "其它", "content": "你慢讲啊。嗯，"},
{"type": "发起", "content": "然后呢？"},
{"type": "讲解", "content": "然后种了杏树。"},
{"type": "发起", "content": "最后呢？"},
{"type": "讲解", "content": "最后还种了桃树。"},
{"type": "发起", "content": "到最后怎么样啊？"},
{"type": "发起", "content": "好，于凯，你能不能连起来把这句话来说一说？“猴子种了”。"},
{"type": "评价", "content": "非常棒，请坐。"},
{"type": "发起", "content": "我们像他一样，用一句话把这个故事，哎，就讲出来了。来，我们一起来试试看。“猴子种了”。"},
{"type": "评价", "content": "瞧，咱们一句话就能把故事的意思讲出来了。小朋友们真能干。"},
{"type": "发起", "content": "好，那我们来看看哦，他为什么什么都没种成呢？我们来读课文的第一段。来，谁来读？"},
{"type": "其它", "content": "那等一下啊。"},
{"type": "发起", "content": "好，你来读。“猴子”。"},
{"type": "其它", "content": "嗯嗯，你先不要坐下，"},
{"type": "发起", "content": "这个“天天”说明什么呀？"}
]

predict = [
  {"type": "发起", "content": "请袁艺喊“开始上课”，声音要响亮啊。"},
  {"type": "其它", "content": "同学们好，请坐。"},
  {"type": "评价", "content": "真棒。"},
  {"type": "讲解", "content": "昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。我们来跟他打打招呼。“猴子”。昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？好，现在用眼睛看，用心记住这个字，“猴子”。"},
  {"type": "发起", "content": "哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。它分别种了什么树呢？谁来说说？"},
  {"type": "发起", "content": "于凯，你来说说看。"},
  {"type": "其它", "content": "你慢讲啊。"},
  {"type": "发起", "content": "嗯，然后呢？"},
  {"type": "讲解", "content": "然后种了杏树。"},
  {"type": "发起", "content": "最后呢？"},
  {"type": "讲解", "content": "最后还种了桃树。"},
  {"type": "发起", "content": "到最后怎么样啊？"},
  {"type": "发起", "content": "好，于凯，你能不能连起来把这句话来说一说？“猴子种了”。"},
  {"type": "评价", "content": "非常棒，请坐。"},
  {"type": "讲解", "content": "我们像他一样，用一句话把这个故事，哎，就讲出来了。来，我们一起来试试看。“猴子种了”。瞧，咱们一句话就能把故事的意思讲出来了。小朋友们真能干。"},
  {"type": "讲解", "content": "好，那我们来看看哦，他为什么什么都没种成呢？我们来读课文的第一段。来，谁来读？那等一下啊。"},
  {"type": "发起", "content": "好，你来读。“猴子”。"},
  {"type": "其它", "content": "嗯嗯，你先不要坐下，这个“天天”说明什么呀？"}
]


In [68]:
find_best_match_expanded_with_results(predict, label)

[{'type': '发起',
  'content': '请袁艺喊“开始上课”，声音要响亮啊。',
  'best_match': {'type': '发起', 'content': '请袁艺喊“开始上课”，声音要响亮啊。'}},
 {'type': '其它',
  'content': '同学们好，请坐。',
  'best_match': {'type': '其它', 'content': '同学们好，请坐。'}},
 {'type': '评价',
  'content': '真棒。',
  'best_match': {'type': '评价', 'content': '真棒。'}},
 {'type': '讲解',
  'content': '昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。我们来跟他打打招呼。“猴子”。昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？好，现在用眼睛看，用心记住这个字，“猴子”。',
  'best_match': {'type': '讲解',
   'content': '昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？'}},
 {'type': '发起',
  'content': '哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。它分别种了什么树呢？谁来说说？',
  'best_match': {'type': '讲解', 'content': '哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。'}},
 {'type': '发起',
  'content': '于凯，你来说说看。',
  'best_match': {'type': '发起', 'content': '它分别种了什么树呢？谁来说说？于凯，你来说说看。'}},
 {'type': '其它',
  'content': '你慢讲啊。',
  'best_match': {'type': '其它', 'content': '你慢讲啊。嗯，'}},
 {'type':

In [20]:

best_match = find_best_match(predict_item, label)

predict_item: {'type': '发起', 'content': '对，我要请只非常棒的小喜鹊。嗯，哎，掌声鼓励。'}
predict_item: {'type': '发起', 'content': '要批评。你等会儿三十秒的话都到我这来。'}
predict_item: {'type': '发起', 'content': '想不读就不读。也不是在那里随便想讲话就讲话。'}
predict_item: {'type': '发起', 'content': '对，'}


In [ ]:
[
  {"type":"其它", "content":"乌鸦说过了，咱们不重复。你不着急吗？意思就是说你会着急的，是不是啊？意思一样啊。"},
  {"type":"评价", "content":"好，很好，掌声鼓励。"},
  {"type":"讲解", "content":"听了三只鸟儿的话，猴子可有想法了。猴子是怎么想的？听了乌鸦的话，猴子一想，你们瞧，可等不及，也等不及，还是等不及。说明这只猴子越来越没有耐心。他越来越没有耐心了，是吧？"},
  {"type":"发起", "content":"现在呀我们拿起书本，把这个故事啊从头到尾再来背一背。用上刚刚老师教你们的，咱们去把故事啊讲流利讲通顺，然后讲的有感情啊。"},
  {"type":"发起", "content":"猴子种了一棵梨树苗，预备起。"},
  {"type":"其它", "content":"猴子，这些东西放下，我们来看一下板书。猴子呀他一开始很勤劳啊，还是种了一棵梨树。"},
  {"type":"讲解","content":"可是当乌鸦来劝说它的时候，它就拔掉了梨树，改种了。正当杏树成活的时候，喜鹊来劝他了。于是猴子又拔掉了，改种了桃树。正当桃树成活的时候，杜鹃又来劝说它了。这时候猴子又拔掉了，改种了。可是有一句农谚叫，樱桃好吃，树难栽，一连几年都没有栽活。到最后这只猴子什么树也没有成。"},
  {"type":"讲解", "content":"对呀，这只猴子什么树也没有种成。除了它没耐心，它还没什么？"},
  {"type":"评价", "content":"你说，说的真好。"},
  {"type":"讲解", "content":"还没有主见，别人说什么他就听什么，没有自己的想法，是不是啊？到最后什么都没有啊。"},
  {"type":"发起", "content":"那么这只小猴呢，也挺可怜的。你想对他说什么？"}
]